In [ ]:
from datasets import load_dataset
from evaluate import load
import transformers
from azure.storage.blob import BlobServiceClient
import pandas as pd
import os
import datasets
import random
from IPython.display import display, HTML
from transformers import AutoTokenizer


In [1]:
# SuperAnnotate
# from superannotate import SAClient
# SAClient(token="<token>")

# # HF
# from huggingface_hub import notebook_login

# notebook_login()

# Azure
# connection_string = "your_connection_string"
# container_name = "your_container_name"
# blob_name = "your_blob_name"
# # Create a blob client using the local file name as the name for the blob
# blob_service_client = BlobServiceClient.from_connection_string(connection_string)
# blob_client = blob_service_client.get_blob_client(container_name, blob_name)


# Fine-tuning a model on a summarization task

In this notebook, we will see how to fine-tune one of the [Azure][https://learn.microsoft.com/en-us/azure/machine-learning/concept-model-catalog?view=azureml-api-2] also found on [Transformers](https://github.com/huggingface/transformers) model  for a summarization task. 

This notebook is built to run with any model checkpoint from the Azure and Transformers Model Hub

In [4]:
# model_checkpoint = "google-t5/t5-small" # For testing locally
# model_checkpoint = 'mistralai/Mistral-7B-v0.1' # final version

## Loading the dataset

### Load dataset already on Datasets
### Get data from Azure blob
### Download data from SA

In [11]:
# loading data from azure blob
# # Download the blob to a local file
# download_file_path = blob_name
# with open(download_file_path, "wb") as download_file:
#     download_file.write(blob_client.download_blob().readall())

# # Load the downloaded data into a pandas DataFrame
# raw_datasets = pd.read_csv(download_file_path)


# raw_datasets = load_dataset("xsum")

# from Superannotate

# raw_datasets = sa.get_annotations(project_name="test", annotation_status="Completed")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
## Metric
# metric = load("rouge")

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [13]:

def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(raw_datasets["train"])

,document,summary,id
0,Test your news knowledge with our 12 days of Christmas news quizzes.\nThis quiz is day five and asks questions about the month of May 2016.,How much do you remember about the news in Wales over the past 12 months?,38366837
1,"Waterman said he was selling the collection, which included 56 Gauge 1 steam models, to fund apprenticeships at the heritage trust he runs.\nTwo of the models sold for £124,000 each, while another went for £86,800.\nHe said the engines had realised what he wanted and he had raised ""not far off"" the amount he hoped.\nA Beyer Goods, which Waterman said was ""the greatest steam railway engine built in miniature"", was one of the models to fetch £120,000.\nThe sale of 32 of the models fetched a total of £627,229, including buyers' premium.\nThe remaining 24 models failed to reach their reserve price and were not sold, but post-sale offers are continuing to come in, the auctioneers said.\nWaterman described the engines, built from scratch for him, as ""the Fabergé eggs of the railway world"".\nHe said he had decided to sell what amounts to around a tenth of his collection in order to raise enough money to secure the future of the Waterman Railway Heritage Trust, which holds his collection of full-size steam engines.\nHe said the pieces he was selling no longer fitted ""into his wider collection"".\n""I never run that stuff any more,"" he said. ""Everything I'm selling is unique. It was all built for me.\n""They are one-offs. They are not toys - they are works of art. That's why they are so pricey.""\nThe sale was carried out by at Dreweatts and Bloomsbury Auctions.\nThe house's steam and model engineering consultant Michael Matthews said: ""The collection is unique.\n""These models were all built from scratch. They were commissioned over a period of 20 years and Mr Waterman got some of the best builders at the time to work for him - few other people would have had the wealth and enthusiasm to do that.""","An auction of part of the model railway collection owned by record producer Pete Waterman has raised more than £600,000.",32331602


You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [15]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [2]:

tokenizer = AutoTokenizer.from_pretrained('t5-small')

prefix = "summarize: "

In [18]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [40]:
# make dataset smaller by half
tokenized_datasets['train'] = tokenized_datasets['train'].shuffle(seed=42).select(range(100))

In [50]:
tokenized_datasets['validation'] = tokenized_datasets['validation'].shuffle(seed=42).select(range(100))

In [49]:
len(tokenized_datasets['validation'])

11332

In [43]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [52]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [53]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [54]:
# The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. 
# We need to define a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [55]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [56]:
trainer.train()

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

TrainOutput(global_step=7, training_loss=3.8635292053222656, metrics={'train_runtime': 264.2064, 'train_samples_per_second': 0.378, 'train_steps_per_second': 0.026, 'train_loss': 3.8635292053222656, 'epoch': 1.0})

In [57]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/herooooooooo/t5-small-finetuned-xsum/commit/0e511a04eb6a30cc650b5c830dd726110593be88', commit_message='End of training', commit_description='', oid='0e511a04eb6a30cc650b5c830dd726110593be88', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoModelForSeq2SeqLM
# example
model = AutoModelForSeq2SeqLM.from_pretrained("herooooooooo/t5-small-finetuned-xsum")

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

c:\Users\kevol\anaconda3\envs\pytorch\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kevol\.cache\huggingface\hub\models--herooooooooo--t5-small-finetuned-xsum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
# make a prediction on a given text

text = "Waterman said he was selling the collection, which included 56 Gauge 1 steam models, to fund apprenticeships at the heritage trust he runs.\nTwo of the models sold for £124,000 each, while another went for £86,800.\nHe said the engines had realised what he wanted and he had raised not far off the amount he hoped.\nA Beyer Goods, which Waterman said was the greatest steam railway engine built in miniature, was one of the models to fetch £120,000.\nThe sale of 32 of the models fetched a total of £627,229, including buyers' premium.\nThe remaining 24 models failed to reach their reserve price and were not sold, but post-sale offers are continuing to come in, the auctioneers said.\nWaterman described the engines, built from scratch for him, as the Fabergé eggs of the railway world.\nHe said he had decided to sell what amounts to around a tenth of his collection in order to raise enough money to secure the future of the Waterman Railway Heritage Trust, which holds his collection of full-size steam engines.\nHe said the pieces he was selling no longer fitted into his wider collection.\nI never run that stuff any more, he said. Everything I'm selling is unique. It was all built for me.\nThey are one-offs. They are not toys - they are works of art. That's why they are so pricey.\nThe sale was carried out by at Dreweatts and Bloomsbury Auctions.\nThe house's steam and model engineering consultant Michael Matthews said: The collection is unique.\nThese models were all built from scratch. They were commissioned over a period of 20 years and Mr Waterman got some of the best builders at the time to work for him - few other people would have had the wealth and enthusiasm to do that."
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
# outputs = model.generate(inputs, max_length=150, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
# print(tokenizer.decode(outputs[0]))



### Uploading custom finetuned model to azure

more or less we follow this [guide][https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-models-from-huggingface?view=azureml-api-2]